In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from model import Model
import loss
import patch_utils
import images_utils
import images_combination

import csv
import json
import ast

from scipy.ndimage import distance_transform_edt, distance_transform_bf, distance_transform_cdt

from tqdm import tqdm
import datetime
from datetime import timedelta
import time

# plot individual samples
from ipywidgets import interact
from IPython.display import display
from ipywidgets import interactive
import ipywidgets as widgets

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as clr
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap

%load_ext autoreload
%autoreload 2

In [ ]:
from os import listdir
from os.path import isfile, join
PATH="../results/csv/EXP_5/"
EXPNAME="EXP_5-2"
onlyfiles = [f for f in listdir(PATH) if isfile(join(PATH, f))]

In [ ]:
filteredFiles=np.array([])
filteredFilesModel=np.array([])
for f in onlyfiles:
    #print(f[0:37])
    if f[0:7]==EXPNAME:
        if f[-9:-4]=='model':
            filteredFilesModel=np.append(filteredFilesModel,f)
        else:
            filteredFiles=np.append(filteredFiles,f)

In [ ]:
def readCSVFiles(filename, modelList):
    csv_file=filename[0:-4]
    csv_file_model=0
    for m in modelList:
        if m[0:-10]==csv_file:
            csv_file_model=m
    return filename, csv_file_model

In [ ]:
def load_csv_model(csv_file_to_import):
    csv_file_model=PATH+csv_file_to_import
    #print(f"Start loading file '{csv_file_model}'")
    #print()
    csv_model_param=[]

    try:
        with open(csv_file_model, newline='') as csvfile:
            csv_model_param=json.load(csvfile)
        #print("Done")
    except IOError:
        print(f"<!-- Loading file '{csv_file_model}' failed -->")

    
    resize_factor=csv_model_param['preprocessing_parameters']['resize']
    patch_nb=csv_model_param['patch_numbers']
    patch_size=csv_model_param['patch_size']

    kernel_dilate_target=np.array(csv_model_param['preprocessing_parameters']['kernel_dilate_target'])
    #print(csv_model_param)
    #print(csv_file_to_import)
    times=np.array(list(csv_model_param['elapsed_time']))
    return resize_factor, patch_nb, patch_size, kernel_dilate_target, times

In [ ]:
def load_csv(csv_file_to_import):
    csv_file = PATH+csv_file_to_import
    csv_data=[]
    #print(f"Start loading file '{csv_file}'")
    #print()

    try:
        with open(csv_file, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                csv_data.append(row)

        #print("This csv contains alignment:")
        #print()
        for k in range(0, len(csv_data), patch_nb):
            csv_anchor_name = csv_data[k]['anchor']
            csv_target_name = csv_data[k]['target']
            #print(f"   {csv_anchor_name} with {csv_target_name}")
        #print()

        #print("Done")
    except IOError:
        print(f"<!-- Loading file '{csv_file_model}' failed -->")
        
    return csv_data

In [ ]:
def verify_img_name(name):
    l=len(name)
    if(l==3):
        return name
    elif(l==2):
        return "0"+name
    elif(l==1):
        return "00"+name
    else:
        return "000"

In [ ]:
def loss_funtion_used(csv_file_to_import):
    csv_file_model=PATH+csv_file_to_import
    csv_model_param=[]

    try:
        with open(csv_file_model, newline='') as csvfile:
            csv_model_param=json.load(csvfile)
        #print("Done")
    except IOError:
        print(f"<!-- Loading file '{csv_file_model}' failed -->")
        
    print(csv_model_param['loss'])

In [ ]:
_, m1 = readCSVFiles(filteredFiles[0], filteredFilesModel)
loss_funtion_used(m1)

In [ ]:
alignmentName=[]
alignmentCount=[]

alignmentScorePositif=[]
alignmentScoreDisruptive=[]

alignmentScorePCount=[]

timesACC=timedelta(seconds=0)

for f in filteredFiles:
    csv_file, csv_file_model = readCSVFiles(f, filteredFilesModel)
    
    resize_factor, patch_nb, patch_size, kernel_dilate_target, times=load_csv_model(csv_file_model)
    csv_data=load_csv(csv_file)
    
    alignmentName.append(f"{verify_img_name(csv_data[0]['anchor'])} -> "
                          +f"{verify_img_name(csv_data[0]['target'])}")
    
    result = ['True'==e['true_positive'] for e in csv_data]
    result=np.asarray(result)
    alignmentCount.append(np.sum(result))
    
    #######################################################
    
    scores=np.array([float(e['score']) for e in csv_data])
    minscore=np.argmin(scores)
    
    argvalid=np.argwhere(result==True)
    alignmentScorePositif.append(minscore in argvalid)
    
    #######################################################
    
    disruptives=np.array([float(e['disruptive_score']) for e in csv_data])
    minscore=np.argmin((disruptives))
    
    argvalid=np.argwhere(result==True)
    alignmentScoreDisruptive.append(minscore in argvalid)    
    
    #######################################################

    argsort_scores=np.argsort(scores)
    argvalid=argvalid.flatten() # all true positive
    argvalid_score=scores[argvalid]
    sorted_avs=np.argsort(argvalid_score)
    sorted_argvalid=argvalid[sorted_avs]
    
    truePositiveSorted=[]
    
    for i in range(len(sorted_argvalid), 0, -1):
        subsec1=set(sorted_argvalid[0:i])
        subsec2=set(argsort_scores[0:i])
        if(subsec1==subsec2):
            truePositiveSorted=list(subsec1)
            break

    alignmentScorePCount.append(truePositiveSorted)
    
    #######################################################
    
    #HH:MM:SS:MS
    hours=timedelta(hours=int(times[0]+times[1]))
    minutes=timedelta(minutes=int(times[3]+times[4]))
    secondes=timedelta(seconds=int(times[6]+times[7]))
    #timesACC+=hours*3600+minutes*60+secondes
    timesACC+=(hours+minutes+secondes)
print(f"Total elapsed time: {timesACC}")

In [ ]:
plt.style.use('ggplot')

argsort=np.argsort(alignmentName)

labels = np.array(alignmentName)[argsort]
indexes = np.arange(len(labels))

rescale = lambda indexes: (indexes - np.min(indexes)) / (np.max(indexes) - np.min(indexes))

plt.barh(indexes, np.array(alignmentCount)[argsort], color='tab:blue')

plt.yticks(indexes, labels)
plt.xticks(np.arange(np.max(alignmentCount)+1))

plt.xlabel('Valid match count', fontsize=16)
plt.ylabel('Anchor -> Target', fontsize=16)
plt.title('Number of valid matches per cadastre alignment pair',fontsize=20)
plt.show()

In [ ]:
plt.style.use('ggplot')

argsort=np.argsort(alignmentName)

labels = np.array(alignmentName)[argsort]
indexes = np.arange(len(labels))

plt.barh(indexes, np.array(alignmentScorePositif)[argsort], color='tab:green')

plt.yticks(indexes, labels)
plt.xticks([0, 1], labels=['False', 'True'])

plt.xlabel('True Positive Match ', fontsize=16)
plt.ylabel('Anchor -> Target', fontsize=16)
plt.title('True positive matches per alignement pair',fontsize=20)
plt.show()

In [ ]:
plt.style.use('ggplot')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,5))

argsort=np.argsort(alignmentName)

labels = np.array(alignmentName)[argsort]
indexes = np.arange(len(labels))

ax1.barh(indexes, np.array(alignmentScorePositif)[argsort], color='tab:green')

ax1.set_yticks(indexes, labels)
ax1.set_xticks([0, 1], labels=['False', 'True'])

ax2.barh(indexes, np.array(alignmentScoreDisruptive)[argsort], color='tab:green')

ax2.set_yticks(indexes, labels)
ax2.set_xticks([0, 1], labels=['False', 'True'])

ax1.set_xlabel('Min Selection ', fontsize=16)
ax2.set_xlabel('Disruptive Selection ', fontsize=16)
ax1.set_ylabel('Anchor -> Target', fontsize=16)
plt.show()

In [ ]:
plt.style.use('ggplot')

argsort=np.argsort(alignmentName)

labels = np.array(alignmentName)[argsort]
indexes = np.arange(len(labels))

countA=[len(x) for x in alignmentScorePCount]

plt.barh(indexes, np.array(countA)[argsort], color='tab:green')

plt.yticks(indexes, labels)
plt.xticks(np.arange(np.max(countA)+1))

plt.xlabel('True Positive Match Count', fontsize=16)
plt.ylabel('Anchor -> Target', fontsize=16)
plt.title('True positive match Count per alignement pair',fontsize=20)
plt.show()